# 프롬포트 엔지니어링 with ChatGPT

In [1]:
!pip install openai

In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

### Chat Completion

In [3]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)
response = client.chat.completions.create(
    model ="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "당신은 아주 친절한 챗봇입니다"
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "화요일 저녁 뭘 먹으면 좋을까?"
                }
            ]
        }
    ],
    response_format={
        "type": "text"
    },
    temperature=0.5,
    max_tokens=2048,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

In [4]:
print(response.choices[0].message.content)

화요일 저녁에는 기분 전환을 위해 다양한 음식을 고려해볼 수 있어요. 몇 가지 추천해드릴게요:

1. **한식**: 비빔밥이나 김치찌개처럼 간단하면서도 맛있는 한식을 즐겨보세요.
2. **양식**: 파스타나 스테이크 같은 양식도 좋은 선택입니다.
3. **일식**: 초밥이나 라멘을 시도해보는 것도 좋습니다.
4. **중식**: 탕수육이나 짬뽕처럼 풍미 있는 중식을 선택해보세요.
5. **가벼운 식사**: 샐러드나 샌드위치처럼 가볍고 건강한 식사를 원하신다면 이런 옵션도 좋습니다.

기분과 상황에 맞춰 선택해 보세요!


### 기사 제목 교정

In [11]:
from openai import OpenAI

def correct_title(query, temperature=0.3):
  client = OpenAI(api_key=OPENAI_API_KEY)
  system_instruction="""
  당신은 편집장입니다. 기자들이 송고한 기사 제목을 교정하세요.

  ###지시사항###
  - 기사의 제목이 명확하고 주제와 잘 맞도록 수정할 것
  - 독자의 관심을 끌 수 있도록 간결하고 임펙트 있는 표현을 사용할 것
  - 비속어, 은어 등은 제거하고 의미가 유지되도록 제목을 교정할 것

  ###출력 형식###
  - 원래 제목: [기사의 원래 제목]
  - 교정 제목:
        [기사의 교정된 제목]
        [기사의 교정된 제목]
        [기사의 교정된 제목]

  ### 예시 ###
  - 원래 제목: "어제 서울에 큰불이 나서 수백명이 대피했다."
  - 교정 제목:
        "서울 대형 화제, 수백명 대피!"
        "서울서 화제, 수백명 대피해!"
        "수백명 대피한 서울 화제"
  """
  user_message=f"""
  다음 제목을 교정해 주세요.
  제목: {query}
  """

  response = client.chat.completions.create(
    model ="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": system_instruction
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": user_message
                }
            ]
        }
    ],
    response_format={
        "type": "text"
    },
    temperature=temperature,
    max_tokens=2048,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
  return response.choices[0].message.content

In [12]:
print(correct_title("컬투쇼 개꿀잼라디오방송에 나올 대박사연을 주목해주세요"))


- 원래 제목: 컬투쇼 개꿀잼라디오방송에 나올 대박사연을 주목해주세요
- 교정 제목:
      "컬투쇼, 흥미진진한 사연 공개!"
      "컬투쇼, 주목할 만한 사연 등장!"
      "컬투쇼, 기대되는 사연 소개!"


### 영단어장 생성

- 팝송 가사에서 단어를 추출하고, 뜻과 다양한 예문 생성

In [28]:
from openai import OpenAI
import json

def extract_eng_word(query, temperature=0.3):

  client = OpenAI(api_key=OPENAI_API_KEY)
  system_instruction = """
당신은 영어 팝송을 이용해 흥미롭고 이해하기 쉬운 방식으로 영어를 가르치는 선생님입니다.

### 처리 단계 ###
1. 주어진 가사에서 영어 단어 5개를 랜덤하게 추출 한다.
2. 각 단어의 의미를 한글로 나타낸다.
3. 각 단어별로 유사한 단어를 함계 소개한다.
4. 유사한 단어에 대한 예문을 작성한다.

### 출력 형식 ###
json 형식으로 출력
- 첫 번째 객체느는 json_list 형식으로 출력
- list 하위에  json_object로 각각의 단어 표현

## 출력 예시 ###
[[
  {{
    "단어":"hello"
    "뜻":"안녕"
    "유사어":[
      "hi":["Hi, I'm Emma", <다음 예시>]
      "good morning": ["Good morning", <다음 예시>]
    ]
  }},
  {{
    "단어":"hello"
    "뜻":"안녕"
    "유사어":[
      "hi":["Hi, I'm Emma", <다음 예시>]
      "good morning": ["Good morning", <다음 예시>]
    ]
  }},
  {{
    "단어":"hello"
    "뜻":"안녕"
    "유사어":[
      "hi":["Hi, I'm Emma", <다음 예시>]
      "good morning": ["Good morning", <다음 예시>]
    ]
  }},
  {{
    "단어":"hello"
    "뜻":"안녕"
    "유사어":[
      "hi":["Hi, I'm Emma", <다음 예시>]
      "good morning": ["Good morning", <다음 예시>]
    ]
  }},
  {{
    "단어":"hello"
    "뜻":"안녕"
    "유사어":[
      "hi":["Hi, I'm Emma", <다음 예시>]
      "good morning": ["Good morning", <다음 예시>]
    ]
  }}
]]
"""
  user_message = f"""
노래 가사: {query}
"""

  response = client.chat.completions.create(
    model ="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": system_instruction
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": user_message
                }
            ]
        }
    ],
    response_format={
        "type": "json_object"
    },
    temperature=temperature,
    max_tokens=2048,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  return json.loads(response.choices[0].message.content)

In [29]:
# 영어 가사 (query) -> 랜덤하게 5개의 단어 추출 ->단어 뜻(한글), 유사어, 유사어의 예문
result = extract_eng_word("""POPULAR!
I'll help you be popular!
You'll hang with the right cohorts,
You'll be good at sports,
Know the slang you've got to know.
So let's start,
'Cause you've got an awfully
long way to go!""")

In [30]:
result

{'json_list': [{'단어': 'popular',
   '뜻': '인기 있는',
   '유사어': {'famous': ['She became famous after her first movie.',
     'He is a famous author.'],
    'well-known': ['The restaurant is well-known for its pasta.',
     'She is well-known in her community.']}},
  {'단어': 'help',
   '뜻': '돕다',
   '유사어': {'assist': ['Can you assist me with this task?',
     'He assisted the elderly woman with her bags.'],
    'aid': ['The charity aids people in need.',
     'She aids her friends with their homework.']}},
  {'단어': 'sports',
   '뜻': '스포츠',
   '유사어': {'athletics': ['She excels in athletics.',
     'Athletics is an important part of school life.'],
    'games': ['The Olympic Games are held every four years.',
     'He enjoys playing video games.']}},
  {'단어': 'slang',
   '뜻': '속어',
   '유사어': {'jargon': ['Medical jargon can be confusing.',
     'He used a lot of technical jargon.'],
    'colloquialism': ['The book is full of colloquialisms.',
     'She often uses colloquialisms in her speech.']

In [32]:
for word_dict in result['json_list']:
  print(word_dict["단어"])
  print(word_dict["뜻"])

  for sim_word in word_dict["유사어"]:
    print(f"{sim_word}: {word_dict['유사어'][sim_word]}")

  print()

popular
인기 있는
famous: ['She became famous after her first movie.', 'He is a famous author.']
well-known: ['The restaurant is well-known for its pasta.', 'She is well-known in her community.']

help
돕다
assist: ['Can you assist me with this task?', 'He assisted the elderly woman with her bags.']
aid: ['The charity aids people in need.', 'She aids her friends with their homework.']

sports
스포츠
athletics: ['She excels in athletics.', 'Athletics is an important part of school life.']
games: ['The Olympic Games are held every four years.', 'He enjoys playing video games.']

slang
속어
jargon: ['Medical jargon can be confusing.', 'He used a lot of technical jargon.']
colloquialism: ['The book is full of colloquialisms.', 'She often uses colloquialisms in her speech.']

start
시작하다
begin: ["Let's begin the meeting.", 'He began his career in 2010.']
commence: ['The event will commence at noon.', 'They commenced the project last year.']



### ReAct 기법

- Reson & Action 기법 : 현재 상황에 대한 통찰 이후 다음 행동에 대한 작성을 유도하는 기법

##### 냉장고를 부탁해

- quety : 냉장공 있는 재료 목록
- return : query로 만든 재료를 활용한 레시피 작성
- [hint] 현재 상황에 대한 분석/파악 + 행동 계획 -> 실행

In [37]:
import json
from openai import OpenAI

def finish_my_fridge(query, temperature=0.3):

    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction = """
당신은 지금부터 '냉장고를 부탁해' 프로그램의 컨셉을 가진 안성재 요리사입니다.
안성재 요리사의 유행어도 함계 사용해서 요리에 재미를 붙일 수 있도록 해줘
예를 들면 이븐하게 읶었어요 등

### 처리 단계 ###
1. 냉장고에 있는 재료를 이해
2. 재료를 활용한 요리 5개 작성
3. 각 요리에 대해 초보자도 이해할 수 있도록 단계별 레시피 작성

### 출력 형식 ###
아래 형식처럼 텍스트로 출력해 주세요:

예시:

요리: 알리오올리오
난이도: 중하
레시피:
1단계: 면을 7분 30초 삶는다
2단계: 마늘과 기름을 3분 볶는다
3단계: 소금을 넣고 밑간을 한다
4단계: 면과 소스를 함께 요리한다

요리: 감자볶음
난이도: 하
레시피:
1단계: 감자를 채 썬다
2단계: 프라이팬에 기름을 두르고 볶는다
3단계: 소금과 후추로 간을 한다
4단계: 투명해질 때까지 익히면 완성

(이런 형식으로 요리 5개를 작성해 주세요.)
"""

    user_message = f"""
냉장고에 있는 재료 목록: {query}
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_instruction
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        response_format={
        "type": "text"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content


In [38]:
result = finish_my_fridge("계란, 양파, 대파, 마늘, 소시지, 치즈, 우유, 감자, 김치, 밥")
print(result)

요리: 김치볶음밥  
난이도: 중하  
레시피:  
1단계: 양파와 대파를 잘게 썬다.  
2단계: 프라이팬에 기름을 두르고 양파, 대파, 다진 마늘을 볶는다.  
3단계: 소시지를 얇게 썰어 넣고 함께 볶는다.  
4단계: 김치를 넣고 3분 정도 볶아준다.  
5단계: 밥을 넣고 고루 섞어가며 볶는다.  
6단계: 마지막으로 치즈를 올려 녹을 때까지 덮어준다.  

요리: 감자전  
난이도: 중  
레시피:  
1단계: 감자를 강판에 갈아 물기를 제거한다.  
2단계: 양파를 잘게 다져 감자와 섞는다.  
3단계: 소금과 후추로 간을 한다.  
4단계: 프라이팬에 기름을 두르고 감자 반죽을 얇게 펴서 부친다.  
5단계: 앞뒤로 노릇하게 익을 때까지 굽는다.  

요리: 계란말이  
난이도: 하  
레시피:  
1단계: 계란을 풀어 소금과 후추로 간을 한다.  
2단계: 대파와 양파를 잘게 썰어 계란에 섞는다.  
3단계: 프라이팬에 기름을 두르고 계란물을 얇게 부어 익힌다.  
4단계: 익은 계란을 돌돌 말아가며 완성한다.  

요리: 마늘 소시지 볶음  
난이도: 하  
레시피:  
1단계: 소시지를 얇게 썬다.  
2단계: 마늘을 편으로 썰어 준비한다.  
3단계: 프라이팬에 기름을 두르고 마늘을 볶아 향을 낸다.  
4단계: 소시지를 넣고 함께 볶는다.  
5단계: 소금과 후추로 간을 맞춘다.  

요리: 치즈 오믈렛  
난이도: 중  
레시피:  
1단계: 계란을 풀어 소금과 후추로 간을 한다.  
2단계: 우유를 약간 넣어 부드럽게 섞는다.  
3단계: 프라이팬에 기름을 두르고 계란물을 부어 익힌다.  
4단계: 계란이 반쯤 익었을 때 치즈를 올린다.  
5단계: 계란을 반으로 접어 치즈가 녹을 때까지 익힌다.  

이븐하게 읶었어요! 맛있게 드세요!


### 면접 질문 생성

In [39]:
from openai import OpenAI

def job_interview(query, temperature=0.3):

    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction = """
당신은 LLM 서비스 개발 분야의 전문가 이면서, 해당 분야의 면접 전문가 입니다.
사용자가 제공한 구인 공고에 근거하여 핵심 질문자들로 지원자를 평가할 수 있어야 합니다.
"""

    user_message = f"""
아래 채용 공고의 직무에 대한 예상 질문과 모범 답안을 작성해 주세요.

--- 가이드 ---
인성면접(태도, 성격 등)과 기술면접으로 2개의 섹션으로 나눠서 작성해 주세요.
각 섹션별로 3개의 질문과 답변을 준비해 주세요.

-- 출력 형식 ---
# 1. 인성 면접
질문1)
답변1)

질문2)
답변2)

질문3)
답변3)

# 2. 기술 면접
질문1)
답변1)

질문2)
답변2)

질문3)
답변3)

--- 채용 공고 ---
query가 지금 html 형식인까 text 형식으로 바꿔서 출력해줘
{query}

"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_instruction
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        response_format={
        "type": "text"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content


In [40]:
query = """
<div class="user_content"><font color="#08d8b7"></font><font color="#28f899"></font><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 16px;"><b>NHN</b></span></span><br></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">NHN은 오늘을 만드는 기술, 세상과 호흡하는 IT 기술로 우리의 일상을 풍요롭게 만듭니다.&nbsp;&nbsp;</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">게임, 기술, 결제, 커머스, 콘텐츠 5대 핵심 사업을 통해 2030년까지 글로벌 테크 컴퍼니로 성장하고자 합니다.</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif;"><br></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt;">___</span>&nbsp;</p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif;">&nbsp;<span style="font-size: 12pt;"></span></p><p style="line-height: 24px;"><span style="font-size: 16px;"><b>VISION&amp;LLM팀을</b></span><span style="font-family: 나눔고딕;"><span style="font-size: 16px;"><b>&nbsp;소개해요</b></span></span><br></p><p style="line-height: 24px;"><span style="font-size: 13.3333px;">VISION&amp;LLM팀은</span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">&nbsp;다양한 챗봇 모델(질의응답 챗봇 모델, 검색 증강 생성 기반 챗봇 모델)을 개발하며 LLM 성능 개선 연구를 진행합니다. 사내외 서비스에 LLM을 적용하기 위해 AI 프로토타입을 개발합니다.</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">현재 한국어 데이터 정제, SFT, PEFT, DPO, Instruction Tuning을 통한 LLM 모델 성능 개선, AI 프로토 타입 제작, 모델 인퍼런스 최적화, RAG 기술 고도화를 담당하고 있으며, 최신 연구를 지속적으로 서베이하고 연구하여 LLM 모델의 성능을 개선하고, 자체 개발한 LLM을 서비스에 적용하는 것을 목표로 하고 있습니다.</span></span></p><div style="font-family: 돋움, Dotum, Helvetica, sans-serif;"><span style="font-size: 12pt;"></span>&nbsp;</div><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><b style="font-family: 나눔고딕; font-size: 16px;">우리 팀은 이렇게 일해요</b></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">·&nbsp;</span></span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">효과적인 문제 해결 및 기술적인 트렌드를 따라가기 위해 다양한 연구와 기술을 적극적으로 반영합니다.</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 13.3333px;">·&nbsp;</span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">다양한 AI 학회와 커뮤니티 소식에 늘 관심을 갖고 최신 연구 및 기술 트렌드를 이해하고 반영하고자 하며, LLM 및 RAG의 성능을 높이기위해 치열하게 고민합니다.&nbsp;</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 13.3333px;">·&nbsp;</span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">연구를 하며 얻는 인사이트나 아이디어를 팀원들과 공유하며 기술적인 도전을 함께 합니다.&nbsp;</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 13.3333px;">·&nbsp;</span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">개발 생산성 및 모델의 성능을 높이기 위해 새로운 기술을 자발적으로 학습하며, 매주 스터디를 통해 최신 트렌드를 공유하고 의견을 나눕니다.</span></span></p><div style="font-family: 돋움, Dotum, Helvetica, sans-serif;"><span style="font-size: 12pt;"></span>&nbsp;</div><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><b style="font-family: 나눔고딕; font-size: 16px;">함께하면 이런 점들이 좋아요</b></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 10pt;">·&nbsp;</span></span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 10pt;">최신 연구를 서베이하고 LLM에 적용해볼 수 있는 기회</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt;">·&nbsp;</span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 10pt;">Multi-Node, Multi-GPU 분산 학습을 경험해볼 수 있는 기회</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt;">·&nbsp;</span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 10pt;">LLM 개발부터 서비스 개발까지 전 과정에 참여해볼 수 있는 환경</span></span></p><div style="font-family: 돋움, Dotum, Helvetica, sans-serif;"><span style="font-size: 12pt;"></span>&nbsp;</div><p style="font-family: 돋움, Dotum, Helvetica, sans-serif;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt;">___</span>&nbsp;</p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif;">&nbsp;<span style="font-size: 12pt;"></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 12pt; color: rgb(68, 68, 68);"><span style="font-weight: 700; font-size: 12pt; color: rgb(0, 0, 0);">이런 업무를 해요 (주요 업무)</span></span><span style="font-size: 12pt;">&nbsp;</span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; color: rgb(68, 68, 68);"><span style="font-size: 13.3333px; color: rgb(0, 0, 0);">·&nbsp;</span></span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">LLM 기술 연구 개발 업무</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 13.3333px;">·&nbsp;</span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">LLM 모델 설계 및 RAG, MoE 등 LLM 연관 기술 연구 개발&nbsp;</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 13.3333px;">·&nbsp;</span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">모델 학습 및 최적화 관련 업무</span></span></p><div style="font-family: 돋움, Dotum, Helvetica, sans-serif;"><p style="line-height: 24px;"><span style="font-size: 12pt;"></span>&nbsp;</p></div><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, ng, 굴림, Gulim, sans-serif; color: rgb(68, 68, 68);"><span style="font-size: 12pt; font-family: 나눔고딕, NanumGothic;"><span style="font-weight: 700; font-size: 12pt; color: rgb(0, 0, 0);">이런 분들을 찾고 있어요 (자격 요건)</span></span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; color: rgb(68, 68, 68);"><span style="font-size: 13.3333px; color: rgb(0, 0, 0);">·&nbsp;</span></span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">관련 분야 석사학위 이상 소지하시거나 개발 경력 2년 이상이신 분</span></span></p><div style="font-family: 돋움, Dotum, Helvetica, sans-serif;"><p style="line-height: 24px;">&nbsp;<span style="font-size: 12pt;"></span></p></div><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, ng, 굴림, Gulim, sans-serif; color: rgb(68, 68, 68);"><span style="font-size: 12pt; font-family: 나눔고딕, NanumGothic;"><span style="font-weight: 700; font-size: 12pt; color: rgb(0, 0, 0);">이런 분이면 더 좋아요 (우대 사항)</span></span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; color: rgb(68, 68, 68);"><span style="font-size: 13.3333px; color: rgb(0, 0, 0);">·&nbsp;</span></span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">세계 최고의 기술을 만들기 위한 열정과 호기심이 있으신 분</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 13.3333px;">·&nbsp;</span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">Python, C/C++ 개발에 능숙하신 분</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 13.3333px;">·&nbsp;</span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">AI 관련 챌린지 상위권 입상 등 수상 실적을 보유하신 분</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 13.3333px;">·&nbsp;</span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">PyTorch, Transformers, DeepSpeed를 사용한 모델 학습 경험을 가지고 계신 분</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 13.3333px;">·&nbsp;</span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">TensorRT, Triton Inference Server를 사용한 모델 최적화 경험을 가지고 계신 분</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 13.3333px;">·&nbsp;</span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">Vector DB, LangChain, Docker를 사용한 개발 경험을 가지고 계신 분</span></span></p><div style="font-family: 돋움, Dotum, Helvetica, sans-serif;"><br></div><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, ng, 굴림, Gulim, sans-serif; color: rgb(68, 68, 68);"><span style="font-size: 10pt; font-family: 나눔고딕, NanumGothic; color: rgb(0, 0, 0);">___</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-size: 12pt;"></span>&nbsp;</p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, ng, 굴림, Gulim, sans-serif; color: rgb(68, 68, 68);"><span style="font-size: 12pt; font-family: 나눔고딕, NanumGothic;"><span style="font-weight: 700; color: rgb(0, 0, 0);">지원서는 이렇게 작성하시면 좋아요</span></span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt;">· 채용 공고에서 요구하는 경험과 역량을 중심으로 기재해주세요.</span>&nbsp;</p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt;">· 해오신 업무를 프로젝트 단위로 잘 정리해서 적어주세요.</span>&nbsp;</p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt;">· 포트폴리오, 노션, 깃허브 등 경험과 역량을 어필할 별도의 자료가 있다면 함께 제출해주세요.</span>&nbsp;</p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt;">· 일부 포지션에서는 포트폴리오 제출이 필수입니다. 확인하시고 꼭 제출해주세요.</span>&nbsp;</p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt;">· 자기소개는 직무 역량을 경험과 잘 연결지어 작성해주시면 좋아요</span>&nbsp;</p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-size: 12pt;"></span>&nbsp;</p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 16px;"><b>NHN으로의 합류 여정</b></span></span><br></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt;">·&nbsp;</span><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">서류전형 &gt; 라이브코딩 &gt; T인터뷰(Tech) &gt; C인터뷰(Career) &gt; 최종합격</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;"><br></span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px; color: rgb(255, 0, 0);">* T인터뷰 진행 시 사전과제가 있습니다. (합격자에 한해, 별도 안내 예정)</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt; color: rgb(68, 68, 68);"><br></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-weight: 700; font-size: 12pt;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 12pt;">꼭 확인해주세요</span>&nbsp;</span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt;">· 남자의 경우, 병역의무를 필하였거나 면제된 분만 지원 가능합니다.</span>&nbsp;</p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt;">· 해외여행에 결격 사유가 없어야 지원 가능합니다.</span>&nbsp;</p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt;">· 국가보훈해당자 및 장애인은 관련 법률에 의거 우대합니다.(증빙서류 제출자에 한함)</span>&nbsp;</p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt;">· 제출서류에 허위기재 또는 기재착오, 구비서류 미제출 등으로 인한 불이익은 응시자 본인의 책임으로 합니다.</span>&nbsp;</p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt;">· 제출서류나 각종 증명서의 기재내용이 허위일 경우, 응시를 무효로 하며 합격을 취소합니다.</span>&nbsp;</p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt;">· 근무지는 판교 삼평동 플레이뮤지엄(Play Museum)입니다.</span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic;"><span style="font-size: 13.3333px;">· 가지고 계신 경험을 충분히 보여주실 수 있도록 학력/경력은 빠짐없이 모두 작성 부탁드립니다.</span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><br></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, ng, 굴림, Gulim, sans-serif; color: rgb(68, 68, 68);"><span style="font-size: 12pt; font-family: 나눔고딕, NanumGothic;"><span style="font-weight: 700; color: rgb(0, 0, 0);">더 알아보기</span></span></span></p><p style="font-family: 돋움, Dotum, Helvetica, sans-serif; line-height: 24px;"><span style="font-family: 나눔고딕, NanumGothic; font-size: 10pt; color: rgb(68, 68, 68);"><span style="font-weight: 700;"><span style="color: rgb(0, 0, 0);">· 공식</span>&nbsp;<span style="color: rgb(255, 0, 0);"><u><a href="https://www.jobkorea.co.kr/List_GI/Co_HomePage_Open.asp?home=https%3A//www.linkedin.com/company/nhncorp&amp;GI_No=44951280" target="_blank"><span style="color: rgb(255, 0, 0);"><u>링크드인</u></span></a></u></span>과&nbsp;<span style="color: rgb(255, 0, 0);"><u><a href="https://www.jobkorea.co.kr/List_GI/Co_HomePage_Open.asp?home=https%3A//www.instagram.com/nhn_careers/&amp;GI_No=44951280" target="_blank"><span style="color: rgb(255, 0, 0);"><u>인스타그램</u></span></a></u></span>,&nbsp;<span style="color: rgb(255, 0, 0);"><u><a href="https://www.jobkorea.co.kr/List_GI/Co_HomePage_Open.asp?home=https%3A//inside.nhn.com/br/index.nhn&amp;GI_No=44951280" target="_blank"><span style="color: rgb(255, 0, 0);"><u>INSIDE NHN</u></span></a></u></span><span style="color: rgb(0, 0, 0);">에서는 더 생생한 소식을 확인할 수 있습니다.</span></span></span>&nbsp;</p><font color="#4bdccd"></font><font color="#6e779a"></font><font color="#c0ffee"></font></div>
"""

print(job_interview(query))

# 1. 인성 면접

질문1) 
NHN의 VISION&LLM팀에서 일하게 된다면, 팀의 연구 및 기술 트렌드를 따라가기 위해 어떤 노력을 기울일 것인가요?

답변1) 
저는 최신 기술 트렌드를 따라잡기 위해 정기적으로 AI 관련 학회 및 커뮤니티에 참여하고, 최신 논문을 읽는 습관을 가지고 있습니다. VISION&LLM팀에 합류하게 된다면, 이러한 습관을 유지하면서 팀 내 스터디 모임에 적극적으로 참여하여 최신 트렌드를 공유하고, 팀원들과의 토론을 통해 깊이 있는 인사이트를 얻고자 합니다. 또한, 새로운 기술을 실험적으로 적용해보는 것을 두려워하지 않고, 이를 통해 얻은 인사이트를 팀과 공유하여 팀의 성장을 도모하겠습니다.

질문2) 
팀 내에서 기술적인 도전을 함께 한다고 했는데, 과거에 팀원들과 협력하여 문제를 해결한 경험을 공유해 주실 수 있나요?

답변2) 
이전 프로젝트에서 대규모 데이터 처리 문제를 해결해야 했던 경험이 있습니다. 당시 팀원들과 함께 다양한 접근 방식을 논의하고, 각자의 강점을 살려 역할을 분담했습니다. 저는 데이터 처리 파이프라인을 최적화하는 역할을 맡았고, 다른 팀원들은 데이터 분석 및 시각화를 담당했습니다. 주기적인 회의를 통해 진행 상황을 공유하고, 발생하는 문제에 대해 함께 해결책을 모색했습니다. 결국, 효율적인 데이터 처리 시스템을 구축하여 프로젝트를 성공적으로 완료할 수 있었습니다.

질문3) 
NHN의 VISION&LLM팀에서 제공하는 기회 중 가장 매력적으로 느껴지는 부분은 무엇인가요?

답변3) 
VISION&LLM팀에서 최신 연구를 서베이하고 LLM에 적용해볼 수 있는 기회가 가장 매력적으로 느껴집니다. AI 분야는 빠르게 발전하고 있으며, 최신 기술을 직접 연구하고 적용해보는 경험은 개인의 성장뿐만 아니라 팀의 발전에도 큰 기여를 할 수 있다고 생각합니다. 또한, Multi-Node, Multi-GPU 분산 학습을 경험할 수 있는 기회도 매우 흥미롭습니다. 이러한 경험은 기술적인 깊이를 더하고, 보다 효율적인 모